### Load tensorflow

In [0]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
#Reset Default graph - Needed only for Jupyter notebook
tf.reset_default_graph()

### Collect Data

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('./prices.csv')

### Check all columns in the dataset

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['symbol','date'], axis=1, inplace = True)


In [38]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set

In [0]:
data = data.head(1000)

### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split
X = data.iloc[:,0:5].values
X = X[:,[0,2,3,4]]
Y = data.iloc[:,1].values

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.30, random_state=1)

In [0]:

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train =  sc.fit_transform(X_train) 
X_test =  sc.fit_transform(X_test) 

In [76]:
X_train.shape


(700, 4)

In [77]:
y_train.shape

(700,)

## Building the graph in tensorflow

1.Define input data placeholders

In [0]:
#Input features
x = tf.placeholder(shape=[None,4],dtype=tf.float32, name='x-input')
#x_n = tf.nn.l2.normalize(x)
#Actual Prices
y_ = tf.placeholder(shape=[None],dtype=tf.float32, name='y-input')

2.Define Weights and Bias

In [0]:
W = tf.Variable(tf.zeros(shape=[4,1]), name="Weights")
b = tf.Variable(tf.zeros(shape=[1]),name="Bias")

3.Prediction

In [0]:
y = tf.add(tf.matmul(x,W),b,name='output')

4.Loss (Cost) Function [Mean square error]

In [0]:
loss = tf.reduce_mean(tf.square(y-y_),name='Loss')

5.GradientDescent Optimizer to minimize Loss [GradientDescentOptimizer]

In [0]:
train_op = tf.train.GradientDescentOptimizer(0.03).minimize(loss)

## Execute the Graph for 100 epochs and observe the loss

In [0]:
#Lets start graph Execution
sess = tf.Session()

# variables need to be initialized before we can use them
sess.run(tf.global_variables_initializer())

#how many times data need to be shown to model
training_epochs = 100

In [96]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, train_loss = sess.run([train_op,loss],feed_dict={ x: X_train, y_: y_train})
    
    if epoch % 10 == 0:
        print ('Training loss at step: ', epoch, ' is ', train_loss)

('Training loss at step: ', 0, ' is ', 3567.6086)
('Training loss at step: ', 10, ' is ', 3567.3276)
('Training loss at step: ', 20, ' is ', 3567.2446)
('Training loss at step: ', 30, ' is ', 3567.2236)
('Training loss at step: ', 40, ' is ', 3567.2158)
('Training loss at step: ', 50, ' is ', 3567.2134)
('Training loss at step: ', 60, ' is ', 3567.2122)
('Training loss at step: ', 70, ' is ', 3567.212)
('Training loss at step: ', 80, ' is ', 3567.2126)
('Training loss at step: ', 90, ' is ', 3567.2136)


### Get the shapes and values of W and b

Hint: Use sess.run(W) to get W.

In [86]:
 sess.run(W)

array([[ 7.5339351e-08],
       [ 1.4165414e-08],
       [-5.3015523e-08],
       [ 2.4026383e-08]], dtype=float32)

In [87]:
sess.run(b)

array([66.45034], dtype=float32)

### Find the Absolute mean square loss difference between training and testing loss.

In [97]:
train_loss

3567.2126

In [91]:
for epoch in range(training_epochs):
            
    #Calculate train_op and loss
    _, test_loss = sess.run([train_op,loss],feed_dict={ x: X_test, y_: y_test})
    
    if epoch % 10 == 0:
        print ('Testing loss at step: ', epoch, ' is ', test_loss)

('Testing loss at step: ', 0, ' is ', 3017.9866)
('Testing loss at step: ', 10, ' is ', 3017.9866)
('Testing loss at step: ', 20, ' is ', 3017.9866)
('Testing loss at step: ', 30, ' is ', 3017.9866)
('Testing loss at step: ', 40, ' is ', 3017.9866)
('Testing loss at step: ', 50, ' is ', 3017.9866)
('Testing loss at step: ', 60, ' is ', 3017.9866)
('Testing loss at step: ', 70, ' is ', 3017.9866)
('Testing loss at step: ', 80, ' is ', 3017.9866)
('Testing loss at step: ', 90, ' is ', 3017.9866)


In [92]:
test_loss

3017.9866

In [98]:
abs(train_loss-test_loss)

549.2261

### Linear Classification using Keras

### Building the simple Neural Network in Keras with one neuron in the dense hidden layer.
#### Use Mean square error as loss function and sgd as optimizer

In [99]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1,input_shape=(4,)))
model.add(tf.keras.layers.Dense(1, activation='softmax'))
model.compile(optimizer='sgd', loss='mse')

Instructions for updating:
Use tf.cast instead.


### Execute the model

In [101]:
model.fit(X_train, y_train, epochs=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
700/700 [==============================] - 0s 293us/sample - loss: 7868.8897
Epoch 2/100
700/700 [==============================] - 0s 38us/sample - loss: 7868.8898
Epoch 3/100
700/700 [==============================] - 0s 35us/sample - loss: 7868.8898
Epoch 4/100
700/700 [==============================] - 0s 33us/sample - loss: 7868.8898
Epoch 5/100
700/700 [==============================] - 0s 34us/sample - loss: 7868.8899
Epoch 6/100
700/700 [==============================] - 0s 37us/sample - loss: 7868.8899
Epoch 7/100
700/700 [==============================] - 0s 35us/sample - loss: 7868.8898
Epoch 8/100
700/700 [==============================] - 0s 38us/sample - loss: 7868.8899
Epoch 9/100
700/700 [==============================] - 0s 36us/sample - loss: 7868.8899
Epoch 10/100
700/700 [==============================] - 0s 37us/sample - loss: 7868.8899
Epoch 11/100
700/700 [==============================] - 0s 38us/sample

### Classification using Keras 

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv('./Iris.csv')

### Splitting the data into feature set and target set

In [0]:
X_iris = iris.drop(['Species','Id'], axis=1)
Y_iris = iris['Species']

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [121]:
Y_iris.value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: Species, dtype: int64

### Divide the dataset into Training and test (70:30)

In [0]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [0]:
Y1 = encoder.fit_transform(Y_iris)

In [0]:
Y2 = pd.get_dummies(Y1)

In [0]:
from sklearn.model_selection import train_test_split
x1_train, x1_test, y1_train, y1_test = train_test_split(X_iris, Y2, test_size=0.2, random_state = 50)

### Model
Build the model with following layers: <br>
1. First dense layer with 10 neurons with input shape 4 (according to the feature set) <br>
2. Second Dense layer with 8 neurons <br>
3. Output layer with 3 neurons with softmax activation (output layer, 3 neurons as we have 3 classes) <br>
4. Use SGD and categorical_crossentropy loss 

In [0]:
model_new = tf.keras.models.Sequential()
model_new.add(tf.keras.layers.Dense(10,input_shape=(4,)))
model_new.add(tf.keras.layers.Dense(8))
model_new.add(tf.keras.layers.Dense(3, activation='softmax'))
model_new.compile(optimizer='sgd', loss='categorical_crossentropy',metrics=['accuracy'])

### Fitting the model and predicting 

In [127]:
model_new.fit(x1_train, y1_train, epochs=1000)

Epoch 1/1000
120/120 [==============================] - 0s 1ms/sample - loss: 2.6520 - acc: 0.1583
Epoch 2/1000
120/120 [==============================] - 0s 90us/sample - loss: 1.3733 - acc: 0.4083
Epoch 3/1000
120/120 [==============================] - 0s 81us/sample - loss: 1.1776 - acc: 0.3750
Epoch 4/1000
120/120 [==============================] - 0s 96us/sample - loss: 1.0834 - acc: 0.4417
Epoch 5/1000
120/120 [==============================] - 0s 87us/sample - loss: 0.9757 - acc: 0.4000
Epoch 6/1000
120/120 [==============================] - 0s 74us/sample - loss: 0.9192 - acc: 0.4250
Epoch 7/1000
120/120 [==============================] - 0s 85us/sample - loss: 0.8400 - acc: 0.5417
Epoch 8/1000
120/120 [==============================] - 0s 66us/sample - loss: 0.7800 - acc: 0.7333
Epoch 9/1000
120/120 [==============================] - 0s 76us/sample - loss: 0.7406 - acc: 0.7667
Epoch 10/1000
120/120 [==============================] - 0s 88us/sample - loss: 0.7050 - acc: 0.8000


In [128]:
scores = model_new.evaluate(x1_test, y1_test)

30/30 [==============================] - 0s 1ms/sample - loss: 0.1762 - acc: 0.9333


### Report Accuracy of the predicted values

In [129]:
print(scores)

[0.17616385221481323, 0.93333334]


In [130]:
print("%s: %.2f%%" % (model_new.metrics_names[1], scores[1]*100))

acc: 93.33%
